# Neural Network with 2D Data
- Neural Network with 4 layers
- 2D data f(x,y) -> R

## 1.Import Required Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom()

## 2. Data Generation

In [2]:
num_data=1000
num_epoch=5000

x = init.uniform(torch.Tensor(num_data,1),-10,10)
y = init.uniform(torch.Tensor(num_data,1),-10,10)
z = x**2 + y**2

x_noise = x + init.normal(torch.FloatTensor(num_data,1),std=0.5)
y_noise = y + init.normal(torch.FloatTensor(num_data,1),std=0.5)
z_noise = x_noise**2 + y_noise**2

data_noise = torch.cat([x_noise,y_noise,z_noise],1)

In [3]:
# data visualization

win_1=viz.scatter(
        X=data_noise,
        opts=dict(
            markersize=5,
            markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=[51,153,255]*np.ones(shape=[num_data,3]))
            )
        )

## 3. Model & Optimizer

In [4]:
model = nn.Sequential(
            nn.Linear(2,20),
            nn.ReLU(),
            nn.Linear(20,10),
            nn.ReLU(),
            nn.Linear(10,5),
            nn.ReLU(),
            nn.Linear(5,5),
            nn.ReLU(),
            nn.Linear(5,1),
        ).cuda()

loss_func = nn.L1Loss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

## 4. Train 

In [5]:
input_data = torch.cat([x,y],1).cuda()
label = z_noise.cuda()
loss_arr =[]

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(input_data))
    loss = loss_func(output,Variable(label))
    loss.backward()
    optimizer.step()
    
    loss_arr.append(loss.cpu().data.numpy()[0])
    
    if i % 100 == 0:
        print(loss)
        data = torch.cat([input_data.cpu(),output.cpu().data],1)

        win_2 =viz.scatter(
                X=data,
                opts=dict(
                markersize=5,
                markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=128*np.ones(shape=[num_data,3]))
            )
    )

Variable containing:
 68.5214
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 63.2572
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 19.6800
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 17.4722
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 15.7753
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 14.4038
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 12.8888
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 11.2582
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 9.3760
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 7.8480
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 7.3190
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 7.0718
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 6.8624
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 6.7326
[torch.cuda.Fl

In [6]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
 1.0006  0.3704
 0.0212 -0.8641
-0.5265 -0.9388
-0.0247  0.3726
-0.1807 -0.5073
-0.7079  0.8594
 1.0520  0.5459
-1.0987  0.2502
-0.0497 -1.1885
-0.6721 -0.3403
 0.4215 -0.7769
 0.1122 -0.2271
 0.7061  0.0737
-0.5351  0.6767
 0.0307 -0.0288
 0.4156  0.9384
-0.6418  0.0994
 0.8648 -0.1796
-1.1547 -0.1620
 1.0124 -0.2104
[torch.cuda.FloatTensor of size 20x2 (GPU 0)]
, Parameter containing:
 0.4415
-0.6126
 0.1387
-1.6824
-1.4579
-0.2661
 0.1088
 0.7412
-0.4832
-1.1438
-0.2886
-0.9381
-1.0335
-1.2684
-0.6048
-0.9223
-1.6081
 0.9622
 0.1651
-0.7356
[torch.cuda.FloatTensor of size 20 (GPU 0)]
, Parameter containing:

Columns 0 to 9 
 0.6421 -0.0411  0.7380 -1.1939 -0.9349  1.3839  0.0909  0.9662  0.3064 -0.9713
 0.1819 -0.0575 -0.0592  0.7220  0.4367  0.1751  0.1911  0.2938 -0.1516  0.5145
-0.0651  0.4594  0.4051  0.7076  0.2063  0.3008  0.3447  0.5018  0.2850  0.5748
 0.0770  0.4897  0.0598  0.3851  0.3911  0.4787  0.5050  0.6577  0.6291  0.0610
-0.1753 -0.2272  0.062

## 5. Visualize Trained Output

In [7]:
data = torch.cat([input_data.cpu(),output.cpu().data],1)

win_2 =viz.scatter(
        X=data,
        opts=dict(
        markersize=5,
        markercolor=np.ndarray(shape=[num_data,3],dtype=float,buffer=128*np.ones(shape=[num_data,3]))
    )
)

## 6. Visualize Loss Graph

In [8]:
x = np.reshape([i for i in range(num_epoch)],newshape=[num_epoch,1])
loss_data = np.reshape(loss_arr,newshape=[num_epoch,1])

win3=viz.line(
    X = x,
    Y = loss_data,
    opts=dict(
        xtickmin=0,
        xtickmax=num_epoch,
        xtickstep=1,
        ytickmin=0,
        ytickmax=20,
        ytickstep=1,
        markercolor=np.random.randint(0, 255, num_epoch),
    ),
)